In [29]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import h5py
import numpy as np
from typing import List, Tuple
from pathlib import Path
import lightgbm as lgb
import pickle

from alber.train import get_vitrine_for_train, train

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. Parameters

In [31]:
base = Path('../../Storage/alber')

dict_features = {
    1: ['stoch_k_price_21_1'],
    29: [
        'stoch_k_price_21_1',
        'Money',
        'ask_size2',
        'ask_spread',
        'bid_ask_spread2',
        'bid_ask_w_spread2',
        'bid_price2',
        'bid_size2',
        'bid_spread',
        'log_return_mean_price',
        'rel_order_count_1_80',
        'rel_order_count_20_40',
        'rel_order_count_40_80',
        'rel_order_count_5_80',
        'rel_price_10_40',
        'rel_price_1_5',
        'rel_price_40_80',
        'rel_price_5_10',
        'rel_size_10_80',
        'rel_size_40_80',
        'rel_stoch_price_21_1_3',
        'rel_volume_ask',
        'rel_volume_ask_bid',
        'rel_volume_ask_bid1',
        'stoch_d_size_42_3',
        'stoch_k_size_14_1',
        'total_volume',
        'volume_imbalance',
        'wap_balance'
    ],
    15: [
        'stoch_k_price_21_1',
        'wap_balance',
        'volume_imbalance',
        'rel_order_count_1_80',
        'rel_price_5_10',
        'log_return_mean_price',
        'bid_spread',
        'ask_spread',
        'Money',
        'rel_stoch_price_21_1_3',
        'rel_volume_ask_bid1',
        'bid_ask_spread2',
        'total_volume',
        'rel_price_1_5',
        'rel_price_40_80'
    ],
}

train_val_ratio = 0.1
test_size = 985_564

# Get train_vitrine (features + time + target)

In [4]:
train_vitrine = get_vitrine_for_train(base)
train_vitrine.shape

(9855646, 61)

In [5]:
train_vitrine.tail(200)

,time,rel_volume_ask_bid1,rel_price_1_10,volume_ask,rel_order_count_1_80,rel_volume_ask,rel_price_1_20,bid_price2,rel_order_count_40_80,rel_size_10_80,...,log_return2,rel_order_count_5_80,rel_price_20_80,stoch_k_price_14_1,Ret_z,rel_price_20_40,stoch_k_price_21_1,total_volume,bid_size2,target
9855446,6479846553,18.841270,0.999813,0.375000,0.975610,-0.500000,0.999643,8249.519531,1.000000,0.072347,...,0.000000,1.170732,0.999600,0.000000,-1.284734,0.999867,0.000000,0.409500,0.028200,-0.000021
9855447,6479846909,7.593014,0.999832,0.617323,0.975610,-0.353805,0.999648,8249.500000,1.000000,0.085312,...,0.000029,0.975610,0.999580,0.000000,-1.389157,0.999854,0.000000,0.775523,0.130000,-0.000353
9855448,6479847848,-0.676903,0.999902,0.310323,0.975610,2.563574,0.999698,8249.519531,1.000000,0.698407,...,0.000092,0.975610,0.999551,0.000000,-1.900326,0.999834,0.000000,1.160320,0.099997,-0.000230
9855449,6479848545,-0.676903,1.000114,0.310323,0.975610,2.563574,0.999912,8249.519531,1.000000,0.787825,...,0.000000,0.975610,0.999548,49.702381,-0.712740,0.999831,30.474451,1.160320,0.099997,-0.000230
9855450,6479848868,-0.997851,1.000281,0.074003,0.975610,-0.927493,1.000103,8250.799805,1.000000,0.802889,...,0.000314,0.975610,0.999540,100.000000,0.751199,0.999824,55.307400,3.152540,0.750000,-0.000279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9855641,6479994731,9.291454,1.000050,1.730785,0.987654,-0.593755,0.999776,8230.799805,1.012346,1.288838,...,-0.000085,0.987654,0.999758,36.049202,-0.350651,0.999734,30.303921,1.780369,0.001000,-0.000138
9855642,6479995854,0.432123,0.999961,1.272737,0.987654,0.020217,0.999701,8229.259766,1.012346,1.376132,...,-0.000187,0.987654,0.999727,23.462606,-1.156316,0.999716,18.886133,1.722537,0.001000,0.000106
9855643,6479996824,-0.744000,0.999961,1.115843,0.987654,-0.939154,0.999701,8229.700195,1.012346,1.376132,...,0.000162,0.987654,0.999727,23.462606,-1.156316,0.999716,18.886133,1.740843,0.375000,0.000038
9855644,6479997863,-0.976074,0.999845,0.266621,0.987654,-0.942089,0.999590,8231.700195,1.012346,1.350261,...,0.000143,0.987654,0.999690,3.958869,-0.853488,0.999694,3.186676,0.891621,0.015000,0.000002


# 1. features_mode == 1

Baseline only for one feature (the best one feature)

# 1.0 Parameters

In [6]:
features_mode = 1
name_exp = 'baseline_1f'

# 1.1 Train

In [9]:
r2_score_test_1f = train(
    train_vitrine[['time', 'target'] + dict_features[features_mode]], 
    name_exp, 
    train_val_ratio, 
    test_size
)

len_train_dates == 7983074, len_val_dates == 887008, len_test_dates == 985564
min_train_dates == 0, min_val_dates == 6706, min_test_dates == 5838738540
max_train_dates == 5838737647, max_val_dates == 5838736765, max_test_dates == 6479998786


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1991: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050921 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 256
[LightGBM] [Info] Number of data points in the train set: 7983074, number of used features: 1
[LightGBM] [Info] Start training from score 0.000001
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [10]:
r2_score_test_1f

0.007133475637148057

# 2. features_mode == 29

Model for 29 feature (features had been obtained from one_factor analysis: delete the worst + smart cut off by correlation)

# 2.0 Parameters

In [11]:
features_mode = 29
name_exp = 'model_of_29f'

# 2.1 Train

In [13]:
r2_score_test_29f = train(
    train_vitrine[['time', 'target'] + dict_features[features_mode]], 
    name_exp, 
    train_val_ratio, 
    test_size
)

len_train_dates == 7983074, len_val_dates == 887008, len_test_dates == 985564
min_train_dates == 1287, min_val_dates == 0, min_test_dates == 5838738540
max_train_dates == 5838737647, max_val_dates == 5838735733, max_test_dates == 6479998786


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1991: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.088473 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6917
[LightGBM] [Info] Number of data points in the train set: 7983074, number of used features: 29
[LightGBM] [Info] Start training from score 0.000001
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[50]	training's rmse: 0.000633747	valid_1's rmse: 0.000637754
[LightGBM] [Warn

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[250]	training's rmse: 0.000627949	valid_1's rmse: 0.000632621
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [14]:
r2_score_test_29f

0.0040940327734526605

# 3. features_mode == 15 - the best model

Model for 15 feature (features had been obtained from multifactor analysis)

# 3.0 Parameters

In [26]:
features_mode = 15
name_exp = 'model_mf_15f'

# 3.1 Train

In [27]:
r2_score_test_15f = train(
    train_vitrine[['time', 'target'] + dict_features[features_mode]], 
    name_exp, 
    train_val_ratio, 
    test_size
)

len_train_dates == 7983074, len_val_dates == 887008, len_test_dates == 985564
min_train_dates == 0, min_val_dates == 1287, min_test_dates == 5838738540
max_train_dates == 5838737647, max_val_dates == 5838737622, max_test_dates == 6479998786


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1991: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.079328 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3677
[LightGBM] [Info] Number of data points in the train set: 7983074, number of used features: 15
[LightGBM] [Info] Start training from score 0.000001
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Li

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[250]	training's rmse: 0.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [28]:
r2_score_test_15f

0.01933963324367216